## Exercise 02. Preprocessing ##

## Imports ##

In [11]:
import pandas as pd

## Task ##

## 1. Download and read ##

1. Загрузите и прочитайте CSV-файл, сделав колонку ID индексом.

In [12]:
df = pd.read_csv('../data/auto.csv', index_col='ID')
df.head()

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN


## 2. Count observations ##

2. Подсчитайте количество наблюдений с помощью метода count().

In [13]:
df.count()

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64

## 3. Drop duplicates ##

3. Удалите дубликаты, учитывая только следующие колонки: CarNumber, Make_n_Model и Fines.
   + Между двумя одинаковыми записями выберите последнюю.
   + Снова проверьте количество наблюдений.

In [14]:
df.drop_duplicates(['CarNumber', 'Make_n_model', 'Fines'], inplace=True, keep='last')
df.count()

CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64

## 4. Missing values ##

4. Работа с отсутствующими значениями.
   + Проверьте, сколько значений отсутствует в каждой колонке.
   + Удалите все колонки с более чем 500 пропущенными значениями, используя аргумент thresh. Проверьте, сколько пропущенных значений осталось в каждой колонке.
   + Замените все отсутствующие значения в колонке Refund на предыдущее значение в этой колонке для данной ячейки. Используйте аргумент method. Проверьте снова количество пропущенных значений в каждой колонке.
   + Замените все отсутствующие значения в колонке Fines на среднее значение этой колонки (без учета пропущенных значений при вычислении среднего). Проверьте количество пропущенных значений.

In [15]:
df.isna().sum()

CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64

In [ ]:
df.dropna(axis=1, thresh=len(df) - 500, inplace=True)
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64

In [17]:
# Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
df['Refund'] = df['Refund'].ffill()
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund           0
Fines           60
dtype: int64

In [18]:
df['Fines'] = df['Fines'].fillna(df['Fines'].mean())
df.isna().sum()

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64

## 5. Make and Model ##

5. Разделите и распарсите Make и Model.
   + Используйте метод apply для разделения и извлечения значений в новые колонки Make и Model.
   + Удалите колонку Make_n_Model.
   + Сохраните датафрейм в JSON-файл auto.json в указанном формате.

In [19]:
df[['Make', 'Model']] = df['Make_n_model'].apply(lambda x: x.split(' ')).apply(pd.Series)
df.drop('Make_n_model', axis=1, inplace=True)
df.head()

,CarNumber,Refund,Fines,Make,Model
ID,,,,,
0,Y163O8161RUS,2.0,3200.0,Ford,Focus
1,E432XX77RUS,1.0,6500.0,Toyota,Camry
2,7184TT36RUS,1.0,2100.0,Ford,Focus
3,X582HE161RUS,2.0,2000.0,Ford,Focus
5,92918M178RUS,1.0,5700.0,Ford,Focus


In [20]:
# df.to_json('../data/auto.json', orient='records', index=False, lines=False)
records = df.to_dict(orient='records')
json_str_list = []
for record in records:
    dict_str = '{'
    for i, (key, value) in enumerate(record.items()):
        if isinstance(value, str):
            value_str = '"' + value.replace('"', '\\"') + '"'
        else:
            value_str = str(value)
            if value_str == 'nan':
                value_str = 'null'
        dict_str += f'"{key}":{value_str}'
        if i < len(record) - 1:
            dict_str += ', '
    dict_str += '}'
    json_str_list.append(dict_str)

json_array_str = '[' + ',\n'.join(json_str_list) + ']'

with open('../data/auto.json', 'w') as f:
    f.write(json_array_str)